In [7]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

using Profile
using PProf
using ProfileSVG

import Base: show

In [8]:
function kalman_filter_graph()
    x_prev_add = AdditionNode()
    
    x_prior = datavar(:x_prior, Normal{Float64})
    add_1   = constvar(:add_1, 1.0)

    connect!(x_prev_add, :in1, x_prior, 1)
    connect!(x_prev_add, :in2, add_1, 1)
    
    noise = constvar(:noise, Normal(0.0, sqrt(200.0)))
    
    add_x_and_noise = AdditionNode()
    
    x = simplerandomvar(:x)
    
    connect!(x_prev_add, :out, x, 1)
    connect!(add_x_and_noise, :in1, x, 2)
    connect!(add_x_and_noise, :in2, noise, 1)
    
    y = datavar(:y, Float64)
    
    connect!(add_x_and_noise, :out, y, 1)
    
    activate!(x_prev_add)
    activate!(add_x_and_noise)
    
    return x_prior, x, y
end

function kalman(data)
    N = length(data)
    
    x_prior, x, y = kalman_filter_graph()
    
    beliefs = Vector{Normal{Float64}}(undef, N)
    updates = Channel{Tuple{Normal{Float64}, Float64}}(Inf) do ch
        while true
            update_data = take!(ch)
            update!(x_prior, update_data[1])
            update!(y, update_data[2])
        end
    end
    
    actor = sync(lambda(Tuple{Int, AbstractBelief},
        on_next = (d) -> begin
        
            index  = d[1]    
            belief = getdata(d[2])
                    
            if index < N
                push!(updates, (belief, data[index + 1]))
            else
                finish!(x_prior)
                finish!(y)
                close(updates)
            end
                
            beliefs[index] = belief
        end,
        on_error = (e) -> println(e)
    ), timeout = 10000)
    
    @async begin
        try
            subscribe!(belief(x) |> enumerate(), actor)
            push!(updates, (Normal(0.0, 1000.0), data[1]))
        catch err 
            println(err)
        end
    end
    
    wait(actor)
    
    return beliefs
end

kalman (generic function with 1 method)

In [9]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [12]:
@btime kalman($data)

  1.082 ms (13663 allocations: 582.94 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=12.027637738287154, σ=14.140721622265264)
 Normal{Float64}(μ=6.544503799492612, σ=9.999500037496876)
 Normal{Float64}(μ=10.055099229126098, σ=8.164693657357805)
 Normal{Float64}(μ=7.832679861108569, σ=7.070891041799029)
 Normal{Float64}(μ=11.359967390055616, σ=6.3244288330249585)
 Normal{Float64}(μ=12.999712026167385, σ=5.773406469256952)
 Normal{Float64}(μ=12.488965617279574, σ=5.34514847952991)
 Normal{Float64}(μ=15.225865340333375, σ=4.999937501171851)
 Normal{Float64}(μ=12.729561113514885, σ=4.713992830503184)
 Normal{Float64}(μ=12.368078676492768, σ=4.472091234310838)
 Normal{Float64}(μ=13.729040505286777, σ=4.263975563874187)
 Normal{Float64}(μ=13.360080027862647, σ=4.082448884373011)
 Normal{Float64}(μ=16.491648512760335, σ=3.922292531398713)
 ⋮
 Normal{Float64}(μ=589.067704689824, σ=0.5827164478148905)
 Normal{Float64}(μ=590.088432977649, σ=0.5822224110578259)
 Normal{Float64}(μ=591.0554593101726, σ=0.5817296287317493)
 N

In [11]:
@time kalman(data)

  0.001639 seconds (13.66 k allocations: 582.922 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=12.027637738287154, σ=14.140721622265264)
 Normal{Float64}(μ=6.544503799492612, σ=9.999500037496876)
 Normal{Float64}(μ=10.055099229126098, σ=8.164693657357805)
 Normal{Float64}(μ=7.832679861108569, σ=7.070891041799029)
 Normal{Float64}(μ=11.359967390055616, σ=6.3244288330249585)
 Normal{Float64}(μ=12.999712026167385, σ=5.773406469256952)
 Normal{Float64}(μ=12.488965617279574, σ=5.34514847952991)
 Normal{Float64}(μ=15.225865340333375, σ=4.999937501171851)
 Normal{Float64}(μ=12.729561113514885, σ=4.713992830503184)
 Normal{Float64}(μ=12.368078676492768, σ=4.472091234310838)
 Normal{Float64}(μ=13.729040505286777, σ=4.263975563874187)
 Normal{Float64}(μ=13.360080027862647, σ=4.082448884373011)
 Normal{Float64}(μ=16.491648512760335, σ=3.922292531398713)
 ⋮
 Normal{Float64}(μ=589.067704689824, σ=0.5827164478148905)
 Normal{Float64}(μ=590.088432977649, σ=0.5822224110578259)
 Normal{Float64}(μ=591.0554593101726, σ=0.5817296287317493)
 N

In [10]:
@time kalman(data)

  3.408731 seconds (9.43 M allocations: 490.182 MiB, 3.70% gc time)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=12.027637738287154, σ=14.140721622265264)
 Normal{Float64}(μ=6.544503799492612, σ=9.999500037496876)
 Normal{Float64}(μ=10.055099229126098, σ=8.164693657357805)
 Normal{Float64}(μ=7.832679861108569, σ=7.070891041799029)
 Normal{Float64}(μ=11.359967390055616, σ=6.3244288330249585)
 Normal{Float64}(μ=12.999712026167385, σ=5.773406469256952)
 Normal{Float64}(μ=12.488965617279574, σ=5.34514847952991)
 Normal{Float64}(μ=15.225865340333375, σ=4.999937501171851)
 Normal{Float64}(μ=12.729561113514885, σ=4.713992830503184)
 Normal{Float64}(μ=12.368078676492768, σ=4.472091234310838)
 Normal{Float64}(μ=13.729040505286777, σ=4.263975563874187)
 Normal{Float64}(μ=13.360080027862647, σ=4.082448884373011)
 Normal{Float64}(μ=16.491648512760335, σ=3.922292531398713)
 ⋮
 Normal{Float64}(μ=589.067704689824, σ=0.5827164478148905)
 Normal{Float64}(μ=590.088432977649, σ=0.5822224110578259)
 Normal{Float64}(μ=591.0554593101726, σ=0.5817296287317493)
 N